In [1]:
import sys, os #imp
sys.path.append('..')
import pandas as pd
import yaml
import backtesting as bt
import matplotlib.pyplot as plt

from tools.instruments import instruments

In [2]:
%load_ext autoreload
%aimport backtesting.system
%aimport backtesting.trader
%aimport tools.quotes
%aimport backtesting.trades
%autoreload 2

In [3]:
plt.ioff()

In [4]:
with open('systems/sample.yaml', encoding='utf-8') as f:
    system1 = yaml.load(f, Loader=yaml.FullLoader)
    print(system1)

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD'], 'from_date': '2019-08-01', 'to_date': None, 'principal': 1000000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.01, 'max_lots': 100, 'commission': 5.98, 'skid': 20, 'allow_pyramiding': False, 'metrics': [['ema20', 'EMA', 'window=20'], ['ema200', 'EMA', 'window=200'], ['min30', 'MIN', 'window=30'], ['max300', 'MAX', 'window=300']], 'entry_rule': {'long': 'ema20>ema200', 'short': None}, 'exit_rule': {'long': 'ema20<ema200', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}


In [5]:
trader = bt.Trader(systems=[system1])
system = trader.systems[0]
trader.run()

매매시작
거래일: 2019-08-01 00:00:00, 시스템: testSystem 3.5375356674194336sec
거래일: 2019-08-02 00:00:00, 시스템: testSystem 0.0009965896606445312sec
거래일: 2019-08-05 00:00:00, 시스템: testSystem 0.0019953250885009766sec
거래일: 2019-08-06 00:00:00, 시스템: testSystem 0.001993894577026367sec
거래일: 2019-08-07 00:00:00, 시스템: testSystem 0.0009970664978027344sec
거래일: 2019-08-08 00:00:00, 시스템: testSystem 0.001995086669921875sec
거래일: 2019-08-09 00:00:00, 시스템: testSystem 0.0009970664978027344sec
거래일: 2019-08-12 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2019-08-13 00:00:00, 시스템: testSystem 0.0009996891021728516sec
거래일: 2019-08-14 00:00:00, 시스템: testSystem 0.001992464065551758sec
거래일: 2019-08-15 00:00:00, 시스템: testSystem 0.0009944438934326172sec
거래일: 2019-08-16 00:00:00, 시스템: testSystem 0.000997304916381836sec
거래일: 2019-08-19 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2019-08-20 00:00:00, 시스템: testSystem 0.0009965896606445312sec
거래일: 2019-08-21 00:00:00, 시스템: testSystem 0.001995086669921875sec


거래일: 2020-03-06 00:00:00, 시스템: testSystem 0.0020232200622558594sec
거래일: 2020-03-09 00:00:00, 시스템: testSystem 0.0009949207305908203sec
거래일: 2020-03-10 00:00:00, 시스템: testSystem 0.0019931793212890625sec
거래일: 2020-03-11 00:00:00, 시스템: testSystem 0.0009989738464355469sec
거래일: 2020-03-12 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2020-03-13 00:00:00, 시스템: testSystem 0.0019683837890625sec
거래일: 2020-03-16 00:00:00, 시스템: testSystem 0.001995086669921875sec
거래일: 2020-03-17 00:00:00, 시스템: testSystem 0.0009968280792236328sec
거래일: 2020-03-18 00:00:00, 시스템: testSystem 0.0020215511322021484sec
거래일: 2020-03-19 00:00:00, 시스템: testSystem 0.000997304916381836sec
거래일: 2020-03-20 00:00:00, 시스템: testSystem 0.0019922256469726562sec
거래일: 2020-03-23 00:00:00, 시스템: testSystem 0.001997709274291992sec
거래일: 2020-03-24 00:00:00, 시스템: testSystem 0.000995635986328125sec
거래일: 2020-03-25 00:00:00, 시스템: testSystem 0.001996278762817383sec
거래일: 2020-03-26 00:00:00, 시스템: testSystem 0.0009911060333251953sec
거래일

거래일: 2020-09-07 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-09-08 00:00:00, 시스템: testSystem 0.002022266387939453sec
거래일: 2020-09-09 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2020-09-10 00:00:00, 시스템: testSystem 0.001995086669921875sec
거래일: 2020-09-11 00:00:00, 시스템: testSystem 0.0029914379119873047sec
거래일: 2020-09-14 00:00:00, 시스템: testSystem 0.001993894577026367sec
거래일: 2020-09-15 00:00:00, 시스템: testSystem 0.001996278762817383sec
거래일: 2020-09-16 00:00:00, 시스템: testSystem 0.0019893646240234375sec
거래일: 2020-09-17 00:00:00, 시스템: testSystem 0.002997159957885742sec
거래일: 2020-09-18 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-09-21 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-09-22 00:00:00, 시스템: testSystem 0.001991748809814453sec
거래일: 2020-09-23 00:00:00, 시스템: testSystem 0.00296783447265625sec
거래일: 2020-09-24 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2020-09-25 00:00:00, 시스템: testSystem 0.0029921531677246094sec
거래일: 2

거래일: 2021-03-19 00:00:00, 시스템: testSystem 0.002969503402709961sec
거래일: 2021-03-22 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2021-03-23 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-03-24 00:00:00, 시스템: testSystem 0.002991914749145508sec
거래일: 2021-03-25 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-03-26 00:00:00, 시스템: testSystem 0.002991914749145508sec
거래일: 2021-03-29 00:00:00, 시스템: testSystem 0.0029921531677246094sec
거래일: 2021-03-30 00:00:00, 시스템: testSystem 0.001995086669921875sec
거래일: 2021-03-31 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2021-04-01 00:00:00, 시스템: testSystem 0.002991914749145508sec
거래일: 2021-04-02 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-04-05 00:00:00, 시스템: testSystem 0.0009975433349609375sec
거래일: 2021-04-06 00:00:00, 시스템: testSystem 0.0030145645141601562sec
거래일: 2021-04-07 00:00:00, 시스템: testSystem 0.0020012855529785156sec
거래일: 2021-04-08 00:00:00, 시스템: testSystem 0.0029630661010742188sec
거래일

거래일: 2022-02-17 00:00:00, 시스템: testSystem 0.0019991397857666016sec
거래일: 2022-02-18 00:00:00, 시스템: testSystem 0.001990079879760742sec
거래일: 2022-02-21 00:00:00, 시스템: testSystem 0.0019683837890625sec
거래일: 2022-02-22 00:00:00, 시스템: testSystem 0.0010218620300292969sec
거래일: 2022-02-23 00:00:00, 시스템: testSystem 0.001996755599975586sec
거래일: 2022-02-24 00:00:00, 시스템: testSystem 0.0009975433349609375sec
거래일: 2022-02-25 00:00:00, 시스템: testSystem 0.0019669532775878906sec
거래일: 2022-02-28 00:00:00, 시스템: testSystem 0.0020220279693603516sec
거래일: 2022-03-01 00:00:00, 시스템: testSystem 0.0009946823120117188sec
거래일: 2022-03-02 00:00:00, 시스템: testSystem 0.001970052719116211sec
거래일: 2022-03-03 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2022-03-04 00:00:00, 시스템: testSystem 0.0009975433349609375sec
거래일: 2022-03-07 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2022-03-08 00:00:00, 시스템: testSystem 0.0009975433349609375sec
거래일: 2022-03-09 00:00:00, 시스템: testSystem 0.001995563507080078sec
거래

In [8]:
system.create_report()

In [11]:
system.trades.log()

[]

In [69]:
system.metrics.loc['2021-06-01':]

AD                           
               ema30      ema5    min30  max300
date                                           
2021-06-01  0.776994  0.776896  0.76960  0.8031
2021-06-02  0.777015  0.777054  0.76960  0.8031
2021-06-03  0.776336  0.773372  0.76655  0.8031
2021-06-04  0.776337  0.774435  0.76655  0.8031
2021-06-07  0.776450  0.775678  0.76655  0.8031
...              ...       ...      ...     ...
2022-04-28  0.734316  0.716942  0.70595  0.7912
2022-04-29  0.732662  0.714195  0.70595  0.7912
2022-05-02  0.730821  0.710847  0.70355  0.7912
2022-05-03  0.729470  0.710531  0.70355  0.7912
2022-05-04  0.729098  0.714921  0.70355  0.7912

[241 rows x 4 columns]

In [70]:
1 >= 1

True

In [30]:
if signal.get('exit_long'):
    print('hi')

In [33]:
'a' in ['a','b','c']

True

In [44]:
a=[1,2,3]

In [45]:
b=[a,[3],'hh']

In [46]:
c=b.copy()

In [47]:
a.append(4)

In [48]:
b

[[1, 2, 3, 4], [3], 'hh']

In [50]:
c.pop(0)

[1, 2, 3, 4]

In [51]:
c

[[3], 'hh']

In [53]:
b.pop(1)

[3]

In [55]:
b

[[1, 2, 3, 4], 'hh']

In [16]:
starts = {} #첫 거래 시작일
for symbol in ['AD']:
    flag = trader.quotes[symbol][['open','high','low','close']].isna().any(axis=1)
    system.signals.loc[flag, symbol] = np.nan
    starts[symbol] = trader.quotes[symbol][~flag].index[0]

In [17]:
starts

{'AD': Timestamp('1987-01-13 00:00:00')}

In [19]:
flag = trader.quotes['AD'].isna().any(axis=1)

In [27]:
system.signals.first_valid_index()

Timestamp('1987-01-13 00:00:00')

In [31]:
system.signals.index[system.signals.index.get_loc(system.signals.first_valid_index())-1]

Timestamp('1987-01-12 00:00:00')

In [38]:
df=df.iloc[1000]

In [47]:
mask = df.isna().groupby('symbol').any() 

In [51]:
mask[~mask]

symbol
BO    False
C     False
HG    False
W     False
Name: 1963-07-22 00:00:00, dtype: bool

In [60]:
instruments['NQ'].tickunit

0.25

In [55]:
while a:
    print(a.pop())

In [66]:
df=trader.quotes['AD']

In [72]:
df.iloc[0][['open','high','low','close']].notna().all()

False

In [73]:
a=[1,2,3,4]

In [74]:
a.pop(2)

3

In [76]:
for i in a:
    a.remove(i)

In [82]:
system.trades.get_on_fires(symbol='AD')

[]

In [84]:
system.trades.get_on_fires()

[]

In [65]:
df=Quotes(quote['AD'], type='single')

In [67]:
df.MA(window=10, inplace=True)

In [68]:
df

field,open,high,low,close,volume,oi,ma30,ma10
date,,,,,,,,
1959-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-04-28,0.71290,0.71660,0.70595,0.70995,112019.0,144283.0,0.741827,0.727695
2022-04-29,0.71035,0.71845,0.70610,0.70870,124501.0,144179.0,0.740832,0.724340
2022-05-02,0.70710,0.70870,0.70355,0.70415,91807.0,140428.0,0.739580,0.721195


In [73]:
df.columns.levels

AttributeError: 'Index' object has no attribute 'levels'

In [75]:
f=quote.columns

In [88]:
len(f.names)

2

In [81]:
d =df.columns

In [87]:
len(d.names)

1